In [46]:
import numpy as np 
import pandas as pd # 
from matplotlib import pyplot as plt
import seaborn as sns

from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from components.data_pipeline import dataPipeline

import warnings
warnings.filterwarnings('ignore')

In [47]:
#TESTES
df_test = pd.read_csv('input/teste1.csv')
df_test, _ = dataPipeline(df_test)
df_test = df_test.sort_values(by='data_semana_comercial')


df_test2 = pd.read_csv('input/teste2.csv')
df_test2, _ = dataPipeline(df_test2)
df_test2 = df_test2.sort_values(by='data_semana_comercial')

Dataframe carregado!
Colunas 'item' e 'semana_comercial' dropadas!
Vendas Guide Shop (GS) dropadas!
Vendas negativas alteradas para zero!
Variaveis categoricas transformadas em numéricas!
coluna 'data_semana_comercial' transformada em datetime!
Dataset de treino para lojas web criado
Dataset de treino para lojas fisicas criado
Dataframe carregado!
Colunas 'item' e 'semana_comercial' dropadas!
Vendas Guide Shop (GS) dropadas!
Vendas negativas alteradas para zero!
Variaveis categoricas transformadas em numéricas!
coluna 'data_semana_comercial' transformada em datetime!
Dataset de treino para lojas web criado
Dataset de treino para lojas fisicas criado


In [48]:
#TREINO
df = pd.read_csv('input/treino.csv')
df, _ = dataPipeline(df)
df = df.sort_values(by='data_semana_comercial')

Dataframe carregado!
Colunas 'item' e 'semana_comercial' dropadas!
Vendas Guide Shop (GS) dropadas!
Vendas negativas alteradas para zero!
Variaveis categoricas transformadas em numéricas!
coluna 'data_semana_comercial' transformada em datetime!
Dataset de treino para lojas web criado
Dataset de treino para lojas fisicas criado


# Prophet

In [49]:
df_prophet = df.copy()
df_prophet.data_semana_comercial = pd.to_datetime(df_prophet.data_semana_comercial)
df_prophet.drop('cod_loja', axis=1, inplace=True)
df_prophet.columns = ['ds', 'y']
df_prophet.head()

,ds,y
60502,2017-01-01,18
32686,2017-01-08,3
7667,2017-01-15,16
38204,2017-01-22,7
76966,2017-01-29,22


In [50]:
model = Prophet()
model.fit(df_prophet)

17:51:27 - cmdstanpy - INFO - Chain [1] start processing
17:51:27 - cmdstanpy - INFO - Chain [1] done processing


In [51]:
df_prophet_test = df_test.copy()
df_prophet_test.drop('cod_loja', axis=1, inplace=True)
df_prophet_test.data_semana_comercial = pd.to_datetime(df_prophet_test.data_semana_comercial)
df_prophet_test.columns = ['ds', 'venda']
df_prophet_test_pred = df_prophet_test.drop('venda', axis=1).copy()

df_prophet_test2 = df_test2.copy()
df_prophet_test2.drop('cod_loja', axis=1, inplace=True)
df_prophet_test2.data_semana_comercial = pd.to_datetime(df_prophet_test2.data_semana_comercial)
df_prophet_test2.columns = ['ds', 'venda']
df_prophet_test_pred2 = df_prophet_test2.drop('venda', axis=1).copy()

In [58]:
forecast1 = model.predict(df_prophet_test_pred)
forecast2 = model.predict(df_prophet_test_pred2)